In [ ]:
import zipfile

with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('archive')


In [3]:
import os
import random
import shutil

# Paths to your original dataset and the destination folders for train and test data
original_dataset_path = 'cell_images'
train_dataset_path = 'train_data'  # Create this folder manually
test_dataset_path = 'test_data'    # Create this folder manually

# Create train and test directories if they don't exist
os.makedirs(train_dataset_path, exist_ok=True)
os.makedirs(test_dataset_path, exist_ok=True)

# List the subdirectories (classes) in the original dataset folder
class_names = ['Parasitized', 'Uninfected']

# Define the ratio of data to be used for testing (e.g., 0.2 for 20%)
test_ratio = 0.3

# Loop through each class
for class_name in class_names:
    class_folder = os.path.join(original_dataset_path, class_name)
    images = os.listdir(class_folder)
    num_images = len(images)
    
    # Calculate the number of images for testing
    num_test_images = int(num_images * test_ratio)
    
    # Randomly select images for testing
    test_images = random.sample(images, num_test_images)
    
    # Move the selected test images to the test folder
    for image in test_images:
        src_path = os.path.join(class_folder, image)
        dst_path = os.path.join(test_dataset_path, class_name, image)
        shutil.move(src_path, dst_path)
    
    # Move the remaining images to the train folder
    remaining_images = [image for image in images if image not in test_images]
    for image in remaining_images:
        src_path = os.path.join(class_folder, image)
        dst_path = os.path.join(train_dataset_path, class_name, image)
        shutil.move(src_path, dst_path)


In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'2.13.0'

In [3]:
img_width, img_height = 64, 64
# Data augmentation (optional but recommended)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='constant'
)
train_generator = train_datagen.flow_from_directory(
    'train_data',
    target_size=(img_width, img_height),
    batch_size=64,
    class_mode='binary'
)

Found 19292 images belonging to 3 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
    'test_data',
    target_size=(img_width, img_height),
    batch_size=64,
    class_mode='binary'
)

Found 8266 images belonging to 2 classes.


In [39]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [5]:
cnn = tf.keras.models.Sequential()

# Convolutional layers with batch normalization and max pooling
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Flatten layer
cnn.add(tf.keras.layers.Flatten())

# Fully connected layers with dropout
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))  # Adding dropout for regularization

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))  # Adding dropout for regularization

# Output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 62, 62, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 29, 29, 64)        256       
 chNormalization)                                                
                                                        

In [8]:
cnn.fit(train_generator, validation_data = test_generator, epochs = 5)


Epoch 1/5
302/302 [==============================] - 104s 342ms/step - loss: -3216703.7500 - accuracy: 0.4994 - val_loss: 11725750.0000 - val_accuracy: 0.5000
Epoch 2/5
223/302 [=====================>........] - ETA: 13s - loss: -77278304.0000 - accuracy: 0.5008

KeyboardInterrupt: 

In [13]:
cnn.save('malaria_augmented_model.h5')

/home/u120663/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
from keras.models import load_model
from numpy import asarray
from matplotlib import pyplot
from numpy.random import randn
from keras.preprocessing.image import ImageDataGenerator,array_to_img,load_img
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img


In [43]:
model=load_model('malaria_augmented_model.h5')

img1=load_img('cell_validation/Parasitized/C39P4thinF_original_IMG_20150622_111326_cell_1.png')
img2=load_img('cell_validation/Uninfected/C58P19thinF_IMG_20150802_122520_cell_109.png')

x1=img_to_array(img1)
x2=img_to_array(img2)

In [44]:
desired_img_shape = (64, 64)
x1_resized = tf.image.resize(x1, desired_img_shape)
x2_resized = tf.image.resize(x2, desired_img_shape)

# Convert them to numpy arrays and reshape to (1, 64, 64, 3)
x1_input = x1_resized.numpy().reshape(( 1,64, 64, 3))
x2_input = x2_resized.numpy().reshape(( 1,64, 64, 3))

# Now you can feed them into the model
X1 = cnn.predict(x1_input)
X2 = cnn.predict(x2_input)







1/1 [==============================] - 0s 24ms/step


In [45]:


if X1==1:
    print(' img1 malaria detected')
elif X1==2:
    print('img1 uninfected')
else:
    print('dont know')
if X2==1:
    print('img 2 malaria detected')
elif X2==0:
    print('img2 uninfected')
else:
    print('dont know')

 img1 malaria detected
img 2 malaria detected
